In [1]:
!pip install scikit-learn pandas numpy matplotlib tensorflow scikit-fuzzy


   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ---------------------------------- ----- 786.4/920.8 kB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 920.8/920.8 kB 8.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import skfuzzy as fuzz
from sklearn.decomposition import PCA


In [4]:
import pandas as pd

# Load LIAR dataset
# Update the paths if needed (e.g., "./data/train.csv")
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("valid.csv")
test_df = pd.read_csv("test.csv")

# Just to be safe, print sample rows
print(train_df.head())

# Retain only the 'statement' and 'label' columns
train_df = train_df[['label', 'statement']]
val_df = val_df[['label', 'statement']]
test_df = test_df[['label', 'statement']]


    2635.json        FALSE  \
0  10540.json    half-true   
1    324.json  mostly-true   
2   1123.json        FALSE   
3   9028.json    half-true   
4  12465.json         TRUE   

  Says the Annies List political group supports third-trimester abortions on demand.  \
0  When did the decline of coal start? It started...                                   
1  Hillary Clinton agrees with John McCain "by vo...                                   
2  Health care reform legislation is likely to ma...                                   
3  The economic turnaround started at the end of ...                                   
4  The Chicago Bears have had more starting quart...                                   

                             abortion    dwayne-bohac  \
0  energy,history,job-accomplishments  scott-surovell   
1                      foreign-policy    barack-obama   
2                         health-care    blog-posting   
3                        economy,jobs   charlie-crist   
4    

KeyError: "None of [Index(['label', 'statement'], dtype='object')] are in the [columns]"

In [5]:
train_df = pd.read_csv("train.csv")
print(train_df.columns)
train_df.head()


Index(['2635.json', 'FALSE',
       'Says the Annies List political group supports third-trimester abortions on demand.',
       'abortion', 'dwayne-bohac', 'State representative', 'Texas',
       'republican', '0', '1', '0.1', '0.2', '0.3', 'a mailer'],
      dtype='object')


,2635.json,FALSE,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,FALSE,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,TRUE,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [6]:
# Mapping multiple classes to binary
def simplify_label(label):
    return 1 if label in ['true', 'mostly-true', 'half-true'] else 0

train_df['label'] = train_df['label'].apply(simplify_label)
val_df['label'] = val_df['label'].apply(simplify_label)
test_df['label'] = test_df['label'].apply(simplify_label)


KeyError: 'label'

In [7]:
columns = [
    'id', 'label', 'statement', 'subject', 'speaker', 'job_title',
    'state', 'party', 'barely_true_counts', 'false_counts', 'half_true_counts',
    'mostly_true_counts', 'pants_on_fire_counts', 'context'
]


In [8]:
train_df = pd.read_csv("train.csv", sep='\t', names=columns, header=None)
val_df = pd.read_csv("valid.csv", sep='\t', names=columns, header=None)
test_df = pd.read_csv("test.csv", sep='\t', names=columns, header=None)


In [10]:
def simplify_label(label):
    try:
        return 1 if str(label).strip().lower() in ['true', 'mostly-true', 'half-true'] else 0
    except:
        return 0  # Default to 0 if something goes wrong (e.g., NaN)

train_df['label'] = train_df['label'].apply(simplify_label)
val_df['label'] = val_df['label'].apply(simplify_label)
test_df['label'] = test_df['label'].apply(simplify_label)


In [11]:
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())
print(test_df['label'].value_counts())


label
0    10249
1       20
Name: count, dtype: int64
label
0    1284
Name: count, dtype: int64
label
0    1270
1      13
Name: count, dtype: int64


In [12]:
train_df = train_df[['label', 'statement']]
val_df = val_df[['label', 'statement']]
test_df = test_df[['label', 'statement']]


In [13]:
import pandas as pd

# Define LIAR dataset column names
columns = [
    'id', 'label', 'statement', 'subject', 'speaker', 'job_title',
    'state', 'party', 'barely_true_counts', 'false_counts', 'half_true_counts',
    'mostly_true_counts', 'pants_on_fire_counts', 'context'
]

# Load all three datasets (Update paths if needed)
train_df = pd.read_csv("train.csv", sep='\t', names=columns, header=None)
val_df = pd.read_csv("valid.csv", sep='\t', names=columns, header=None)
test_df = pd.read_csv("test.csv", sep='\t', names=columns, header=None)

# Convert multiclass labels to binary: 1 = true/mostly-true/half-true, 0 = everything else
def simplify_label(label):
    try:
        return 1 if str(label).strip().lower() in ['true', 'mostly-true', 'half-true'] else 0
    except:
        return 0  # fallback in case of NaN or unexpected format

train_df['label'] = train_df['label'].apply(simplify_label)
val_df['label'] = val_df['label'].apply(simplify_label)
test_df['label'] = test_df['label'].apply(simplify_label)

# Keep only relevant columns
train_df = train_df[['label', 'statement']]
val_df = val_df[['label', 'statement']]
test_df = test_df[['label', 'statement']]

# Print samples to verify
print("Train sample:\n", train_df.head())
print("\nLabel distribution:\n", train_df['label'].value_counts())


Train sample:
    label statement
0      0       NaN
1      0       NaN
2      0       NaN
3      0       NaN
4      0       NaN

Label distribution:
 label
0    10249
1       20
Name: count, dtype: int64


In [3]:
# Combine all texts to fit tokenizer
all_texts = pd.concat([train_df['statement'], val_df['statement'], test_df['statement']])

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(all_texts)

# Convert to sequences
max_len = 40
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df['statement']), maxlen=max_len)
X_val = pad_sequences(tokenizer.texts_to_sequences(val_df['statement']), maxlen=max_len)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df['statement']), maxlen=max_len)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(train_df['label'])
y_val = le.transform(val_df['label'])
y_test = le.transform(test_df['label'])

# Convert to categorical (one-hot)
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_val_cat = tf.keras.utils.to_categorical(y_val)
y_test_cat = tf.keras.utils.to_categorical(y_test)


NameError: name 'train_df' is not defined

In [21]:
pip install fcmeans


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement fcmeans (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for fcmeans


In [20]:
# 1. Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score
from fcmeans import FCM
from sklearn.preprocessing import StandardScaler

# 2. Combine all text data to fit tokenizer
all_texts = pd.concat([train_df['statement'], val_df['statement'], test_df['statement']])

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(all_texts)

# 3. Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(train_df['statement'])
X_val_seq = tokenizer.texts_to_sequences(val_df['statement'])
X_test_seq = tokenizer.texts_to_sequences(test_df['statement'])

# 4. Padding
max_length = 50
X_train = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# 5. Labels
y_train = train_df['label'].values
y_val = val_df['label'].values
y_test = test_df['label'].values

# 6. BiLSTM Model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 7. Train
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val)
)

# 8. Evaluate
pred_probs = model.predict(X_test)
y_pred_initial = (pred_probs > 0.5).astype(int).reshape(-1)

print("\n🔍 BiLSTM Performance:")
print(classification_report(y_test, y_pred_initial))

# 9. 🔁 Fuzzy C-Means Post-processing

# Use outputs of last hidden dense layer (64D features)
extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
X_test_features = extractor.predict(X_test)

# Normalize features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test_features)

# Run Fuzzy C-Means clustering
fcm = FCM(n_clusters=2)
fcm.fit(X_test_scaled)
fcm_labels = fcm.predict(X_test_scaled)  # 0 or 1

# Determine which cluster is more "real" based on majority vote with BiLSTM predictions
from scipy.stats import mode
real_cluster = mode(fcm_labels[y_pred_initial == 1])[0][0]
y_pred_fcm_refined = np.where(fcm_labels == real_cluster, 1, 0)

# 10. Final evaluation
print("\n🧠 After Fuzzy C-Means refinement:")
print(classification_report(y_test, y_pred_fcm_refined))
print("Initial Accuracy (BiLSTM):", accuracy_score(y_test, y_pred_initial))
print("Refined Accuracy (BiLSTM + FCM):", accuracy_score(y_test, y_pred_fcm_refined))


ModuleNotFoundError: No module named 'fcmeans'

NameError: name 'model' is not defined

In [25]:
# 1. Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import mode

# # 2. Combine all text data to fit tokenizer
# all_texts = pd.concat([train_df['statement'], val_df['statement'], test_df['statement']])

# tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
# tokenizer.fit_on_texts(all_texts)

# # 3. Convert text to sequences
# X_train_seq = tokenizer.texts_to_sequences(train_df['statement'])
# X_val_seq = tokenizer.texts_to_sequences(val_df['statement'])
# X_test_seq = tokenizer.texts_to_sequences(test_df['statement'])

# 2. Combine all text data to fit tokenizer (ensure strings)
all_texts = pd.concat([train_df['statement'], val_df['statement'], test_df['statement']]).astype(str)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(all_texts)

# 3. Convert text to sequences (ensure strings)
X_train_seq = tokenizer.texts_to_sequences(train_df['statement'].astype(str))
X_val_seq = tokenizer.texts_to_sequences(val_df['statement'].astype(str))
X_test_seq = tokenizer.texts_to_sequences(test_df['statement'].astype(str))


# 4. Padding
max_length = 50
X_train = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# 5. Labels
y_train = train_df['label'].values
y_val = val_df['label'].values
y_test = test_df['label'].values

# 6. BiLSTM Model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 7. Train
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val)
)

# 8. Evaluate
pred_probs = model.predict(X_test)
y_pred_initial = (pred_probs > 0.5).astype(int).reshape(-1)

print("\n🔍 BiLSTM Performance:")
print(classification_report(y_test, y_pred_initial))

# 9. 🔁 KMeans Post-processing
# Use outputs of last hidden dense layer (64D features)
extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
X_test_features = extractor.predict(X_test)

# Normalize features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test_features)

# Run KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_labels = kmeans.fit_predict(X_test_scaled)

# Determine which cluster is more "real" based on majority vote with BiLSTM predictions
# real_cluster = mode(kmeans_labels[y_pred_initial == 1])[0][0]

real_cluster = mode(kmeans_labels[y_pred_initial == 1], keepdims=True).mode[0]
y_pred_kmeans_refined = np.where(kmeans_labels == real_cluster, 1, 0)

# 10. Final evaluation
print("\n🧠 After KMeans refinement:")
print(classification_report(y_test, y_pred_kmeans_refined))
print("Initial Accuracy (BiLSTM):", accuracy_score(y_test, y_pred_initial))
print("Refined Accuracy (BiLSTM + KMeans):", accuracy_score(y_test, y_pred_kmeans_refined))


Epoch 1/5
161/161 [==============================] - 7s 26ms/step - loss: 0.0525 - accuracy: 0.9933 - val_loss: 9.1601e-04 - val_accuracy: 1.0000
Epoch 2/5
161/161 [==============================] - 4s 22ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 1.0917e-04 - val_accuracy: 1.0000
Epoch 3/5
161/161 [==============================] - 4s 22ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 9.3514e-05 - val_accuracy: 1.0000
Epoch 4/5
161/161 [==============================] - 4s 23ms/step - loss: 4.3159e-04 - accuracy: 1.0000 - val_loss: 9.5963e-06 - val_accuracy: 1.0000
Epoch 5/5
41/41 [==============================] - 1s 6ms/step

🔍 BiLSTM Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1270
           1       0.80      0.62      0.70        13

    accuracy                           0.99      1283
   macro avg       0.90      0.81      0.85      1283
weighted avg       0.99      0.99      0.99      1283


c:\Users\tamra\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\tamra\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(



🧠 After KMeans refinement:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1270
           1       0.85      0.85      0.85        13

    accuracy                           1.00      1283
   macro avg       0.92      0.92      0.92      1283
weighted avg       1.00      1.00      1.00      1283

Initial Accuracy (BiLSTM): 0.9945440374123149
Refined Accuracy (BiLSTM + KMeans): 0.9968823070927514


In [26]:
import pickle

# 1. Save the BiLSTM model
model.save('bilstm_model.h5')

# 2. Save the KMeans model
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

# 3. Save the Tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# 4. Save the StandardScaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


c:\Users\tamra\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
